# Работа с инструментами. Выбор для торговли

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from tinkoff.invest import Client

In [ ]:
from app import AppConfig

# запрашиваем все инструменты
with Client(AppConfig.TOKEN) as client:
    instruments = client.instruments.shares()

In [ ]:
i_list = [
    # # 'DELI',
    # # 'ETLN',
    # # 'EUTR',
    # 'GEMC',
    # 'GLTR',
    # 'MTLR',
    # # 'RNFT',
    # # 'SPBE',
    # 'SVAV',
    # 'UWGN',

    # 'SPBE',
    'GEMC',
    'HNFG',
    'UWGN',
    'GLTR',
    # 'EUTR',
    'POLY',
    'ASTR',
    # 'RNFT',
    'QIWI',
]

instrument = None
for instr in instruments.instruments:
    if instr.ticker in i_list and not instr.short_enabled_flag:
        print(f"{instr.ticker} - short={instr.short_enabled_flag}")


In [ ]:
# распечатать конкретный
instrument = None
for instr in instruments.instruments:
    if instr.ticker == 'ETLN':
        instrument = instr

print(instrument)
print(instrument.short_enabled_flag)

In [ ]:
from tinkoff.invest import Share

# фильтруем те, по которым можно вести торги
filtered = list(filter(lambda i: i.buy_available_flag and
                                 i.exchange not in ['spb_close', 'otc_ncc'], instruments.instruments))

for ins in filtered:
    ins: Share
    print(f"{ins.ticker} ({ins.figi}), {ins.name}, {ins.sector}, {ins.trading_status}, {ins.exchange}")

print()
print(len(filtered), 'штук')

In [ ]:
from app.helper import q2f
from tinkoff.invest import CandleInterval, HistoricCandle
from datetime import datetime, timedelta

# человеко понятные большие числа
def human_format(num):
    num = float('{:.3g}'.format(num))
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    return '{}{}'.format('{:f}'.format(num).rstrip('0').rstrip('.'), ['', 'K', 'M', 'B', 'T'][magnitude])

# считаем волатильность
def calculate_volatility(figi, n_days=365) -> dict:
    with Client(AppConfig.TOKEN) as client:
        # Получаем текущую дату и дату n_days назад
        end_date = datetime.now()
        start_date = end_date - timedelta(days=n_days)

        candles_src = client.market_data.get_candles(
            figi=figi,
            from_=start_date,
            to=end_date,
            interval=CandleInterval.CANDLE_INTERVAL_DAY
        ).candles


        candles = []
        ps = []
        vs = []
        first_open = None
        last_close = None
        for c in candles_src:
            c: HistoricCandle
            open_ = q2f(c.open)
            if first_open is None:
                first_open = open_
            if open_ == 0:
                continue
            last_close = q2f(c.close)
            p = (q2f(c.high) - q2f(c.low)) / open_
            candles.append({
                'o': open_,
                'c': q2f(c.close),
                'h': q2f(c.high),
                'l': q2f(c.low),
                'd': q2f(c.high) - q2f(c.low),
                'p': p,
                'v': c.volume,
            })

            vs.append(c.volume)
            ps.append(p)

        if not ps:
            return {}

        return {
            'open': first_open,
            'close': last_close,
            'ps': round(sum(ps) / len(ps), 2),
            'vs': human_format(sum(vs)),
        }

# calculate_volatility('TCS009046502')

In [ ]:
# собираем вместе данные по отфильтрованным инструментам и волатильности
my_list = []
for ins in filtered:
    ins: Share
    my_list.append({
        'ticker': ins.ticker,
        'figi': ins.figi,
        'name': ins.name,
        'sector': ins.sector,
        'exchange': ins.exchange,
        'lot': ins.lot,
        'inc': q2f(ins.min_price_increment),
        'short': ins.short_enabled_flag,
    } | calculate_volatility(ins.figi))


In [ ]:
import pandas as pd

# Создание DataFrame из массива словарей
df = pd.DataFrame(my_list)

df

In [ ]:
# фильтруем данные
filtered_df = df.query('close * lot < 800 and ps>=0.04 and lot==1')

filtered_df

In [ ]:
filtered_df['close'].sum()